In [4]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.preprocessing import OneHotEncoder
import re

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('../../data/interim/data_nun_cleared(2_of_2).csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19737 entries, 0 to 19736
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                19737 non-null  int64  
 1   ID  объявления            19737 non-null  int64  
 2   Ремонт                    19737 non-null  object 
 3   Площадь комнат, м2        19737 non-null  object 
 4   Балкон                    19737 non-null  object 
 5   Окна                      19737 non-null  object 
 6   Санузел                   19737 non-null  object 
 7   Можно с детьми/животными  19737 non-null  object 
 8   Дополнительно             19465 non-null  object 
 9   Высота потолков, м        19737 non-null  float64
 10  Лифт                      19737 non-null  object 
 11  Мусоропровод              19737 non-null  object 
 12  Ссылка на объявление      19737 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 2.0+ MB


In [8]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [9]:
df['Ремонт'].value_counts()

Ремонт
Косметический    8410
Евроремонт       8160
Дизайнерский     3031
Без ремонта       136
Name: count, dtype: int64

In [10]:
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(df[['Ремонт']])

In [11]:
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Ремонт']))

In [12]:
# выкидываю столбец "без ремонта", чтобы не плодить лишние данные
encoded_df.drop('Ремонт_Без ремонта', inplace=True, axis=1)

In [13]:
encoded_df.columns = ['rem_diz', 'rem_evro', 'rem_kosm']
df_final = pd.concat([df['ID  объявления'], encoded_df], axis=1)
df_final.rename(columns={'ID  объявления': 'id'}, inplace=True)

In [14]:
# смотрим, какие символы нас ждут в площади
set(''.join(df['Площадь комнат, м2'].unique().tolist()))

{' ',
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '²',
 'м',
 '—'}

In [15]:
def get_rooms_size(text: str):
    # меняем все запятые на точки для корректного отображения чисел
    # но если точки есть, то считаем запятые разделителями
    
    # работа с дебильными значениями типо 18,13,10    
    if '.' not in text and ' ()+\-/—' in text:
        text = text.replace(',', '.')
    # убираем лишнюю фигню
    for char in 'м²':
        text = text.replace(char, '')
    # все возможные разделители текста
    delimiters = r'[ ()+\-/—,]'
    result = re.split(delimiters, text)
    result = [item for item in result if item]
    return sum((map(float, result)))

In [16]:
# кто-то указал номер телефона в площади... (берем инфу из площади)
df.loc[7136, 'Площадь комнат, м2'] = '20'

In [17]:
df_final['rooms_size'] = df['Площадь комнат, м2'].apply(get_rooms_size)

In [18]:
df['Балкон'].value_counts()

Балкон
Нет                       6630
Балкон (1)                6336
Лоджия (1)                5032
Балкон (1), Лоджия (1)     610
Лоджия (2)                 537
Балкон (2)                 423
Балкон (3)                  45
Лоджия (3)                  42
Балкон (2), Лоджия (2)      21
Балкон (1), Лоджия (2)      21
Балкон (2), Лоджия (1)      16
Лоджия (4)                   5
Балкон (4)                   4
Балкон (3), Лоджия (1)       4
Балкон (1), Лоджия (3)       4
Балкон (2), Лоджия (3)       3
Балкон (1), Лоджия (4)       2
Балкон (3), Лоджия (3)       1
Балкон (4), Лоджия (4)       1
Name: count, dtype: int64

In [19]:
def balcony_values(description):
    balcony_match = re.search(r'Балкон \((\d+)\)', description)
    loggia_match = re.search(r'Лоджия \((\d+)\)', description)
    balcony = int(balcony_match.group(1)) if balcony_match else 0
    loggia = int(loggia_match.group(1)) if loggia_match else 0
    return pd.Series([balcony, loggia])

In [20]:
df_final[['balcony', 'logia']] = df['Балкон'].apply(balcony_values)

In [21]:
df['Окна'].value_counts()

Окна
Во двор            14374
На улицу и двор     3246
На улицу            2117
Name: count, dtype: int64

In [22]:
def view_encode(view):
    if view == 'Во двор':
        return pd.Series([1, 0])
    elif view == 'На улицу':
        return pd.Series([0, 1])
    else:
        return pd.Series([1, 1])

In [23]:
df_final[['dvor_view', 'street_view']] = df['Окна'].apply(view_encode)

In [24]:
def toilet_values(description):
    merged_match = re.search(r'Совмещенный \((\d+)\)', description)
    split_match = re.search(r'Раздельный \((\d+)\)', description)
    merged = int(merged_match.group(1)) if merged_match else 0
    split = int(split_match.group(1)) if split_match else 0
    return pd.Series([merged, split])

In [25]:
df_final[['merged_toilet', 'split_toilet']] = df['Санузел'].apply(toilet_values)

In [26]:
df['Можно с детьми/животными'].value_counts()

Можно с детьми/животными
Можно с детьми                       8540
Можно с детьми, Можно с животными    6075
Ни с кем нельзя!                     4915
Можно с животными                     207
Name: count, dtype: int64

In [27]:
def allow_encode(allow):
    if allow == 'Можно с детьми':
        return pd.Series([1, 0])
    elif allow == 'Можно с животными':
        return pd.Series([0, 1])
    elif allow == 'Можно с детьми, Можно с животными':
        return pd.Series([1, 1])
    else:
        return pd.Series([0, 0])

In [28]:
df_final[['kids_allowed', 'pets_allowed']] = df['Можно с детьми/животными'].apply(allow_encode)

In [29]:
df['Дополнительно'].fillna('', inplace=True)

In [30]:
def encode_features(equipment_list):
    equipment_items = equipment_list.split(', ')
    features = [
        'Мебель в комнатах', 'Мебель на кухне', 'Ванна', 'Душевая кабина', 
        'Стиральная машина', 'Кондиционер', 'Посудомоечная машина', 
        'Телевизор', 'Холодильник', 'Интернет', 'Телефон'
    ]
    encoded_features = {feature: (1 if feature in equipment_items else 0) for feature in features}
    return pd.Series(encoded_features)

In [31]:
df_final[['furn_in_rooms', 'furn_at_kitchen', 'bath',
        'shower', 'washing', 'aircon', 'dishwaser',
        'tv', 'freezer', 'internet', 'phone']] = df['Дополнительно'].apply(encode_features)

In [32]:
# убираем выбросы
df.loc[df['Высота потолков, м'] > 150, 'Высота потолков, м'] = df['Высота потолков, м'] / 100
df.loc[df['Высота потолков, м'] > 15, 'Высота потолков, м'] = df['Высота потолков, м'] / 10

In [33]:
# убираем жилье хоббита
df.loc[df['Высота потолков, м'] == 1.2, 'Высота потолков, м'] = df['Высота потолков, м'].median()

In [34]:
# и великана
df.loc[df['Высота потолков, м'] == 12.8, 'Высота потолков, м'] = df['Высота потолков, м'].median()

In [35]:
df_final['ceiling_height'] = df['Высота потолков, м']

In [36]:
def lift_values(description):
    pass_match = re.search(r'Пасс \((\d+)\)', description)
    gruz_match = re.search(r'Груз \((\d+)\)', description)
    pass_lift = int(pass_match.group(1)) if pass_match else 0
    gruz_lift = int(gruz_match.group(1)) if gruz_match else 0
    return pd.Series([pass_lift, gruz_lift])

In [37]:
df_final[['pass_lift', 'gruz_lift']] = df['Лифт'].apply(lift_values)

In [38]:
df_final['trashprovod'] = df['Мусоропровод'].apply(lambda x: 1 if x == 'Да' else 0)

In [39]:
# разбираемся с выбросами
df_final.loc[df_final['pass_lift'] > 10, 'pass_lift'] = df_final['pass_lift'] / 10
df_final.loc[df_final['gruz_lift'] == 12, 'gruz_lift'] = 1

In [42]:
df_final['link'] = df['Ссылка на объявление']

In [45]:
df_final.head()

,id,rem_diz,rem_evro,rem_kosm,rooms_size,balcony,logia,dvor_view,street_view,merged_toilet,...,dishwaser,tv,freezer,internet,phone,ceiling_height,pass_lift,gruz_lift,trashprovod,link
0,271271157,1.00,0.00,0.00,140.00,0,0,1,1,2,...,1,1,1,1,1,3.00,4,1,1,https://www.cian.ru/rent/flat/271271157
1,271634126,1.00,0.00,0.00,95.00,0,0,1,1,2,...,1,1,1,1,0,3.50,1,1,0,https://www.cian.ru/rent/flat/271634126
2,271173086,0.00,1.00,0.00,116.00,0,0,1,1,3,...,1,1,1,1,1,3.20,1,0,1,https://www.cian.ru/rent/flat/271173086
3,272197456,0.00,1.00,0.00,95.00,0,0,1,1,3,...,1,1,1,1,1,3.20,1,0,1,https://www.cian.ru/rent/flat/272197456
4,273614615,0.00,1.00,0.00,38.00,0,0,1,1,2,...,1,1,1,1,0,3.90,1,1,1,https://www.cian.ru/rent/flat/273614615


In [46]:
# экспортируем
df_final.to_csv('../../data/interim/data_encoded(2_of_2).csv')